In [1]:
import os
from os import path
import pandas as pd

### Build results dataframe

In [2]:
N = 24
tls = [0.5, 0.6, 0.7]
seeds = [i for i in range(1, 4)]
steps = [(i + 1) * 5000 for i in range(N)]

In [3]:
results_list_tl = []
performance_path_tl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/models/planning_results'
name_tl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=False-learnedcost=False-lambdatl={tl}' + \
       '-seed={seed}-novaluestep{step}.model.log'

for tl in tls:
    for seed in seeds:
        for step in steps:
            file_name = path.join(performance_path_tl, name_tl.format(tl=tl, seed=seed, step=step))
            if os.path.isfile(file_name):
                with open(file_name) as f:
                    lines = f.readlines()
                    for line in lines:
                        splits = line.split("|")
                        ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                        time = int(splits[1].split(":")[1].strip())
                        distance = int(splits[2].split(":")[1].strip())
                        success_or_failure = int(splits[3].split(":")[1].strip())
                        df_line = {
                            'lambda_tl': tl,
                            'seed': seed,
                            'step': step,
                            'ep': ep,
                            'time': time,
                            'distance': distance,
                            'result': success_or_failure
                        }
                        results_list_tl.append(df_line)

In [5]:
results_df_tl = pd.DataFrame(results_list_tl)
cols_tl = ['lambda_tl', 'seed', 'step', 'ep', 'time', 'distance', 'result']
results_df_tl = results_df_tl[cols_tl]

In [6]:
results_list_bl = []
performance_path_bl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/models_v12/planning_results/'
name_bl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-' + \
          'lambdal=0.2-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=0-learnedcost=1-' + \
          'seed={seed}-novaluestep{step}.model.log'
for seed in seeds:
    for step in steps:
        file_name = path.join(performance_path_bl, name_bl.format(seed=seed, step=step))
        if os.path.isfile(file_name):
            with open(file_name) as f:
                lines = f.readlines()
                for line in lines:
                    splits = line.split("|")
                    ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                    time = int(splits[1].split(":")[1].strip())
                    distance = int(splits[2].split(":")[1].strip())
                    success_or_failure = int(splits[3].split(":")[1].strip())
                    df_line = {
                        'seed': seed,
                        'step': step,
                        'ep': ep,
                        'time': time,
                        'distance': distance,
                        'result': success_or_failure
                    }
                    results_list_bl.append(df_line)

In [7]:
results_df_bl = pd.DataFrame(results_list_bl)
cols_bl = ['seed', 'step', 'ep', 'time', 'distance', 'result']
results_df_bl = results_df_bl[cols_bl]

### Filter dataframe

In [8]:
# Set filter variables for target_lane results
tl = 0.6
seed = 2
step = 80000
result = 0  # 1 for successes, 0 for failures

In [21]:
# Perform filter on target_lane results
print(f'List of {"success" if result else "failure"} episodes for lambda_tl: {tl}, seed: {seed}, step: {step}:')
filtered_list_tl = results_df_tl[(results_df_tl.lambda_tl == tl) &
                                 (results_df_tl.seed == seed) &
                                 (results_df_tl.step == step) &
                                 (results_df_tl.result == result)]['ep'].values
print(len(filtered_list_tl))
print(filtered_list_tl)

List of failure episodes for lambda_tl: 0.6, seed: 2, step: 80000:
119
[  2  14  15  25  29  31  33  39  41  44  46  50  60  73  77  84  93  95
  99 116 118 119 122 125 131 132 135 136 137 143 151 152 164 167 170 171
 172 179 181 192 203 208 214 217 225 226 227 229 233 234 239 255 262 272
 276 278 280 281 289 296 298 307 308 313 314 315 320 323 324 332 333 336
 342 345 353 354 356 366 369 373 386 389 390 400 404 409 416 417 423 424
 433 437 438 439 456 459 461 462 464 465 471 472 473 476 482 488 501 509
 513 517 519 521 524 525 528 532 547 550 561]


In [28]:
# Set filter variables for baseline results
seed = 3
step = 80000
result = 0  # 1 for successes, 0 for failures

In [29]:
# Perform filter on baseline results
print(f'List of {"success" if result else "failure"} episodes for baseline - seed: {seed}, step: {step}:')
filtered_list_bl = results_df_bl[(results_df_bl.seed == seed) &
                                 (results_df_bl.step == step) &
                                 (results_df_bl.result == result)]['ep'].values
print(len(filtered_list_bl))
print(filtered_list_bl)

List of failure episodes for baseline - seed: 3, step: 80000:
135
[  5  13  18  19  29  32  33  40  48  51  55  57  59  63  74  78  85  89
  91  93  99 100 104 108 123 124 128 130 136 143 151 152 154 158 163 165
 167 171 172 177 179 184 186 187 190 191 195 196 200 206 209 212 214 216
 219 222 228 235 241 242 244 246 253 254 257 263 265 278 286 290 296 297
 301 307 308 311 321 325 329 331 332 337 341 342 343 344 345 353 355 361
 364 367 371 373 377 380 389 393 399 403 404 408 412 413 414 415 417 427
 430 435 436 439 440 448 452 475 478 484 486 487 490 493 508 511 518 522
 527 531 536 538 540 547 548 551 558]
